# Laboratorio 5: RAG Agent

## Parte 0: Instalación de dependencias

En este laboratorio, explorarás la creación e implementación de un agente de Retrieval Augmented Generation (RAG). Un agente RAG combina la capacidad de un modelo de lenguaje para generar texto con la habilidad de recuperar información relevante de una base de conocimiento externa.

A lo largo de este laboratorio, realizarás los siguientes pasos clave:

1.  **Implementación de herramientas**: Desarrollarás funciones para procesar, guardar y buscar información en la base vectorial.
2.  **Creación de agentes**: Construirás agentes especializados (memoria e investigación).
3.  **Implementación del supervisor**: Diseñarás un agente supervisor para dirigir las consultas al agente adecuado.
4.  **Orquestación con LangGraph**: Utilizarás LangGraph para definir el flujo de interacción entre los agentes.
5.  **Pruebas**: Validarás el funcionamiento del agente RAG con diferentes consultas.

Instalación de dependencias

In [ ]:
%pip install langchain-openai openai pinecone sentence-transformers wikipedia langgraph langgraph-supervisor langchain_community langchain-core

## Parte 1: Inicialización del modelo de lenguaje, embeddings y conexión con Pinecone

Inicializa un modelo de lenguaje de OpenAI usando la clase ChatOpenAI.

Configura un modelo de embeddings de Hugging Face. Usa el modelo "sentence-transformers/all-MiniLM-L6-v2".

Conéctate a Pinecone utilizando su clave de API desde userdata.get('pinecone_api_key').

Crea un índice vectorial en Pinecone llamado "general-info" con las siguientes características:
- Dimensión: 384
- Métrica: "cosine"
- Especificación de servidor: tipo ServerlessSpec, nube "aws", región "us-east-1".

In [ ]:
# Celda para Colab: usa los nombres exactos de secrets y crea el índice "general-info"
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import os

# -------------------------
# 1) Leer secrets (NOMBRES EXACTOS)
# -------------------------
OPENAI_API_TOKEN = userdata.get('OPENAI_API_TOKEN')   # nombre exacto pedido
HUGGING_API_KEY     = userdata.get('HUGGING_API_KEY') # nombre exacto pedido
PINECONE_API_KEY    = userdata.get('PINECONE_API_KEY')# nombre exacto pedido

# Validaciones claras
if not OPENAI_API_TOKEN:
    raise ValueError("Falta 'OPENAI_API_TOKEN' en userdata. Añadí la secret con ese nombre en Colab.")
if not HUGGING_API_KEY:
    raise ValueError("Falta 'HUGGING_API_KEY' en userdata. Añadí la secret con ese nombre en Colab.")
if not PINECONE_API_KEY:
    raise ValueError("Falta 'PINECONE_API_KEY' en userdata. Añadí la secret con ese nombre en Colab.")

# Exportar a env vars que algunas librerías esperan
os.environ['OPENAI_API_KEY'] = OPENAI_API_TOKEN
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGING_API_KEY

# -------------------------
# 2) Inicializar ChatOpenAI
# -------------------------
# Se usa la clase ChatOpenAI tal como pide la consigna.
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0, openai_api_key=OPENAI_API_TOKEN)

# -------------------------
# 3) Inicializar embeddings HF (all-MiniLM-L6-v2)
# -------------------------
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
print(f"Iniciando SentenceTransformer {EMBED_MODEL_NAME} (esto descarga el modelo la 1ra vez)...")
sbert = SentenceTransformer(EMBED_MODEL_NAME)
# wrapper de LangChain (opcional para uso con LangChain)
hf_embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)

# -------------------------
# 4) Conectar a Pinecone
# -------------------------
pinecone = Pinecone(api_key=PINECONE_API_KEY)

# Parámetros del índice pedidos por la consigna
index_name = "general-info"
dimension = 384
metric = "cosine"
serverless_spec = ServerlessSpec(cloud="aws", region="us-east-1")

# -------------------------
# 5) Crear índice: varios intentos para compatibilidad con SDKs distintos
# -------------------------
existing_indexes = pinecone.list_indexes()
if index_name in existing_indexes:
    print(f"Índice '{index_name}' ya existe en Pinecone.")
else:
    created = False
    last_exception = None

    # Intento A: argumento 'serverless' (tu intento original)
    try:
        pinecone.create_index(name=index_name, dimension=dimension, metric=metric, serverless=serverless_spec)
        created = True
        print("Índice creado usando 'serverless' argument.")
    except TypeError as e:
        last_exception = e
        # Intento B: argumento 'spec' (algunas versiones usan 'spec' en lugar de 'serverless')
        try:
            pinecone.create_index(name=index_name, dimension=dimension, metric=metric, spec=serverless_spec)
            created = True
            print("Índice creado usando 'spec' argument.")
        except TypeError as e2:
            last_exception = e2
            # Intento C: firma posicional básica (nombre, dimensión, métrica) — puede crear índice sin serverless
            try:
                pinecone.create_index(index_name, dimension, metric)
                created = True
                print("Índice creado usando firma posicional (sin serverless/spec).")
                print("ATENCIÓN: en este último caso la configuración ServerlessSpec NO se aplicó; confirma en la consola de Pinecone si necesitas cambiar el tipo de índice.")
            except Exception as e3:
                last_exception = e3

    if not created:
        # Si no se pudo crear, mostramos error informativo con el último exception
        raise RuntimeError("No se pudo crear el índice con las firmas probadas. Último error:\n" + repr(last_exception))

# -------------------------
# 6) Obtener handle del índice (compatibilidad con distintas versiones)
# -------------------------
index = None
try:
    index = pinecone.index(index_name)   # forma común
except Exception:
    try:
        index = pinecone.Index(index_name)  # alternativa de algunas versiones
    except Exception as e:
        print("No se pudo obtener handle del índice con pinecone.index ni pinecone.Index. Error:", e)
        index = None

print("Hecho. ChatOpenAI inicializado, embeddings cargados y Pinecone listo.")
print("Handle del índice obtenido?:", index is not None)


## Parte 2: Procesamiento de texto y almacenamiento de embeddings en Pinecone

Implementa la función process_and_upsert_files(data: str) que:
- Reciba un texto como entrada.
- Genere su embedding con embedding_model.embed_query(data).
- Guarde el embedding en Pinecone junto con el texto original dentro de la metadata.
- Devuelva un mensaje indicando cuántos fragmentos fueron insertados.

In [ ]:
from uuid import uuid4
from typing import Any

def process_and_upsert_files(data: str) -> str:
    """
    1) Recibe un texto (data).
    2) Genera su embedding usando embedding_model.embed_query(data).
       - Si no hay variable `embedding_model` en globals, intenta usar `hf_embeddings`.
    3) Inserta en Pinecone el vector junto con la metadata que contiene el texto original.
    4) Devuelve un mensaje indicando cuántos fragmentos fueron insertados.
    """
    # Validaciones básicas (esperamos que index e index_name existan en el scope global)
    if "index" not in globals() or globals().get("index") is None:
        raise RuntimeError("No se encontró el handle `index`. Asegurate de crear/obtener el índice Pinecone antes.")
    if "index_name" not in globals() or globals().get("index_name") is None:
        raise RuntimeError("No se encontró `index_name` en el entorno. Define index_name = 'general-info' por ejemplo.")

    # Resolver el objeto de embeddings (se requiere método embed_query)
    embedding_model = globals().get("embedding_model") or globals().get("hf_embeddings")
    if embedding_model is None:
        raise RuntimeError("No hay un modelo de embeddings disponible (ni 'embedding_model' ni 'hf_embeddings').")

    # 1) Generar embedding usando la API requerida: embed_query
    emb = embedding_model.embed_query(data)

    # Normalizar al formato lista (Pinecone espera listas JSON-serializables)
    try:
        emb_list = emb.tolist()  # si es numpy array
    except Exception:
        emb_list = list(emb)     # si es iterador / lista ya

    # 2) Preparar payload para upsert: (id, vector, metadata)
    _id = str(uuid4())
    metadata = {"text": data}  # guardamos el texto original en metadata
    upsert_items = [(_id, emb_list, metadata)]

    # 3) Upsert en Pinecone (manejando variantes de firma del SDK)
    try:
        # intento más común
        index.upsert(vectors=upsert_items)
    except TypeError:
        try:
            # alternativa posible en otras versiones
            index.upsert(items=upsert_items)
        except Exception as e:
            raise RuntimeError("Error al upsert en Pinecone: " + str(e))
    except Exception as e:
        # otros errores (conectividad, keys, etc.)
        raise RuntimeError("Error al upsert en Pinecone: " + str(e))

    # 4) Retornar mensaje con cuántos fragmentos se insertaron (aquí 1 por llamada)
    inserted = len(upsert_items)
    return f"Insertados {inserted} fragmento(s) en el índice '{globals().get('index_name')}'."

Ejemplo rápido de uso (en la celda siguiente, tras ejecutar la función):

In [ ]:
msg = process_and_upsert_files("La capital de Francia es Paris.")
print(msg)

## Parte 3: Búsqueda semántica en Pinecone con la función search_and_fetch

Implementa la función search_and_fetch(query_text: str, top_k: int = 3) que:
- Genere un embedding de la consulta.
- Busque los top_k resultados más similares en Pinecone usando index.query().
- Devuelva los resultados incluyendo la metadata con el texto original.

In [ ]:
import numpy as np
from typing import List, Dict, Any

def search_and_fetch(query_text: str, top_k: int = 3) -> List[Dict[str, Any]]:
    """
    Genera embedding, valida/normaliza el vector y busca los top_k matches en Pinecone.
    Devuelve lista de dicts con keys: 'id', 'score', 'metadata'.
    """
    # 1) Resolver modelo de embeddings
    emb_model = globals().get("embedding_model") or globals().get("hf_embeddings") or globals().get("sbert")
    if emb_model is None:
        raise RuntimeError("No hay modelo de embeddings disponible ('embedding_model', 'hf_embeddings' o 'sbert').")

    # 2) Generar embedding (soporta sbert o wrappers con embed_query/embed)
    if emb_model is globals().get("sbert"):
        q_emb = emb_model.encode([query_text], convert_to_numpy=True)[0]
    else:
        try:
            q_emb = emb_model.embed_query(query_text)
        except Exception:
            q_emb = emb_model.embed([query_text])[0]

    # 3) Normalizar a python floats y validar
    q_arr = np.asarray(q_emb).ravel()
    q_list = [float(x) for x in q_arr.astype(float).tolist()]

    # Diagnóstico mínimo (opcional - coméntalo si no querés prints)
    print("Embedding length:", len(q_list), "| first values:", q_list[:6])

    # Comprobar NaN/Inf y dimensión
    if any(np.isnan(x) for x in q_list):
        raise ValueError("El embedding contiene NaN.")
    if any(np.isinf(x) for x in q_list):
        raise ValueError("El embedding contiene Inf.")
    expected_dim = globals().get("dimension", 384)
    if len(q_list) != expected_dim:
        raise ValueError(f"Dimensión del embedding ({len(q_list)}) distinta de la dimensión esperada ({expected_dim}).")

    # 4) Intentar llamadas a index.query con distintas firmas (primero la que funcionó antes)
    last_exc = None
    resp = None

    try_signatures = [
        ("vector_kw",   lambda v: index.query(vector=v, top_k=top_k, include_metadata=True)),
        ("queries_kw",  lambda v: index.query(queries=[v], top_k=top_k, include_metadata=True)),
        ("query_vector_kw", lambda v: index.query(query_vector=v, top_k=top_k, include_metadata=True)),
        ("positional",  lambda v: index.query(v, top_k)),
        ("single_vector_kw", lambda v: index.query(v, top_k=top_k, include_metadata=True)),
    ]

    for name, fn in try_signatures:
        try:
            # Intentar la firma
            resp = fn(q_list)
            print(f"Query: signature '{name}' OK")
            break
        except Exception as e:
            # Guardar último error y seguir probando
            print(f"Query: signature '{name}' falló -> {type(e).__name__}: {e}")
            last_exc = e
            resp = None
            continue

    if resp is None:
        # Ninguna firma funcionó -> relanzar último error (verás el traceback)
        raise last_exc

    # 5) Normalizar la respuesta de Pinecone a formato consistente
    matches_out: List[Dict[str, Any]] = []

    # helper para procesar lista de matches (cada match puede ser dict o objeto)
    def _process_matches(raw_matches):
        out = []
        for m in raw_matches:
            if isinstance(m, dict):
                mid = m.get("id")
                mscore = m.get("score")
                mmeta = m.get("metadata")
            else:
                mid = getattr(m, "id", None)
                mscore = getattr(m, "score", None)
                mmeta = getattr(m, "metadata", None)
            out.append({"id": mid, "score": mscore, "metadata": mmeta})
        return out

    # Caso 1: dict con 'results' (serverless v2)
    if isinstance(resp, dict) and "results" in resp and resp["results"]:
        results0 = resp["results"][0] or {}
        raw_matches = results0.get("matches", [])
        matches_out = _process_matches(raw_matches)
        return matches_out

    # Caso 2: objeto con attribute 'results' (puede ser None)
    if hasattr(resp, "results") and resp.results:
        # resp.results puede ser lista-like; intentar extraer matches del primer elemento
        try:
            first = resp.results[0]
            # first puede ser dict-like o un objeto con .matches
            if isinstance(first, dict):
                raw_matches = first.get("matches", [])
            else:
                raw_matches = getattr(first, "matches", []) or (first.get("matches", []) if hasattr(first, "get") else [])
            matches_out = _process_matches(raw_matches)
            return matches_out
        except Exception:
            # continuar a otros casos si algo raro pasa
            pass

    # Caso 3: respuesta con atributo matches directo
    if hasattr(resp, "matches") and getattr(resp, "matches") is not None:
        raw_matches = getattr(resp, "matches")
        matches_out = _process_matches(raw_matches)
        return matches_out

    # Caso 4: dict con 'matches' en la raíz
    if isinstance(resp, dict) and "matches" in resp:
        raw_matches = resp.get("matches", [])
        matches_out = _process_matches(raw_matches)
        return matches_out

    # Fallback: devolver la respuesta cruda para inspección si no reconocemos la estructura
    return [{"raw_response": resp}]

Ejemplo de uso (ejecutalo tras definir la función):

In [ ]:
results = search_and_fetch("¿Cuál es la capital de Francia?", top_k=3)
for r in results:
    print(r.get("id"), r.get("score"), r.get("metadata", {}).get("text"))


## Parte 4: Creación de un agente con memoria utilizando herramientas personalizadas y el patrón ReAct

Implementar un agente de memoria que pueda guardar información (memorias) y consultarla a través de dos herramientas personalizadas.

1. Define una herramienta add_memory(query: str) decorada con @tool, cuya función sea:
    - Llamar a process_and_upsert_files(query) para guardar información en la base vectorial.
    - Retornar un mensaje de éxito (“Memory Saved successfully”) o, en caso de error, un mensaje de fallo.

2. Define una herramienta search_memory(query: str) decorada con @tool, que:
    - Utilice la función search_and_fetch(query_text=query) para recuperar información.
    - Devuelva los resultados obtenidos.

3. Crea un mensaje de sistema (prompt) llamado retriever_agent_prompt que describa el propósito del agente.

4. Define una lista de herramientas que contenga ambas funciones (add_memory y search_memory).

5. Crea el agente de memoria usando el patrón ReAct, mediante create_react_agent(), pasando como parámetros:
    - El modelo de lenguaje (llm_model)
    - Las herramientas (ToolNode(tools_list))
    - Un nombre identificativo ("Memory_Agent")
    - El prompt de sistema (retriever_agent_prompt)

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt.chat_agent_executor import create_react_agent
from langgraph.prebuilt import ToolNode
import traceback

# asegurar llm_model
llm_model = globals().get("llm_model") or globals().get("chat")
if llm_model is None:
    raise RuntimeError("No se encontró 'llm_model' ni 'chat'. Inicializalo antes de crear el agente.")

def _norm_text(t: str) -> str:
    """Normaliza texto para comparación simple (minúsculas, espacios)."""
    return " ".join(t.strip().lower().split())

@tool
def add_memory(query: str) -> str:
    """
    Guarda `query` en la base vectorial si no existe ya.
    - Comprueba duplicados con search_and_fetch(query, top_k=10) comparando texto normalizado exacto.
    - Si ya existe un fragmento con el mismo texto normalizado, retorna 'Memory already exists'.
    - Si no existe, llama a process_and_upsert_files(query) y retorna 'Memory Saved successfully' o un mensaje de fallo.
    """
    try:
        # 1) comprobar duplicado exacto normalizado
        try:
            resultados = search_and_fetch(query, top_k=10) or []
        except Exception as e:
            # si la comprobación falla, imprimimos y seguimos para intentar agregar (evitar bloqueo por error temporal)
            print("add_memory: warning en search_and_fetch al comprobar duplicado:", e)
            traceback.print_exc()
            resultados = []

        qnorm = _norm_text(query)
        for r in resultados:
            meta = r.get("metadata") or {}
            candidate_text = meta.get("text") or meta.get("snippet") or ""
            if _norm_text(candidate_text) == qnorm:
                return "Memory already exists"

        # 2) no hay duplicado -> insertar
        process_and_upsert_files(query)
        return "Memory Saved successfully"
    except Exception as e:
        # imprimir traceback para diagnóstico (no silenciamos)
        print("add_memory - error al guardar memory:", e)
        traceback.print_exc()
        return f"Failed to save memory: {e}"

@tool
def search_memory(query: str):
    """
    Recupera memorias relevantes para `query` usando search_and_fetch y devuelve la lista resultante.
    """
    # no capturamos excepciones aquí: el agente las verá si ocurren
    return search_and_fetch(query_text=query, top_k=5)

# Prompt del sistema
retriever_agent_prompt = (
    "Eres Memory_Agent. Tu trabajo es almacenar y recuperar memorias del usuario.\n"
    "Dispones de dos herramientas:\n"
    " - add_memory(query: str): guarda una memoria (usa process_and_upsert_files). Debe evitar duplicados exactos.\n"
    " - search_memory(query: str): recupera memorias relevantes (usa search_and_fetch).\n\n"
    "Instrucciones:\n"
    " - Si el usuario pide guardar algo, usa add_memory y confirma con 'Memory Saved successfully' o 'Memory already exists'.\n"
    " - Si el usuario pide recordar o consultar algo, usa search_memory y resume los resultados indicando las fuentes (metadata.text).\n"
)

# Reconstruir herramientas y agente
tools_list = [add_memory, search_memory]
tools_node = ToolNode(tools_list)

# crear/recrear el agente (firma con prompt= en tu entorno)
memory_agent = create_react_agent(llm_model, tools_node, prompt=retriever_agent_prompt, name="Memory_Agent")

# Ensure the name attribute exists and is correct for the supervisor
if not hasattr(memory_agent, 'name') or memory_agent.name is None or memory_agent.name == "LangGraph":
    print(f"Warning: Setting 'name' attribute on memory_agent to 'Memory_Agent'.")
    memory_agent.name = "Memory_Agent"


print("Tools actualizadas y agente 'Memory_Agent' recreado correctamente.")

Ejemplo de uso (ejecutalo tras definir la función):

In [ ]:
import time
import traceback
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import ToolMessage

def call_agent(agent, prompt: str):
    state_in = { "messages": [HumanMessage(content=prompt)] }
    result_state = agent.invoke(state_in)

    msgs = result_state["messages"]

    # capturar tool usada
    tools_used = [m for m in msgs if isinstance(m, ToolMessage)]
    tool_name = tools_used[-1].name if tools_used else None

    # respuesta del agente
    ai_msgs = [m for m in msgs if isinstance(m, AIMessage)]
    output = ai_msgs[-1].content if ai_msgs else msgs[-1].content

    return {
        "response": output,
        "tool_used": tool_name
    }


# memorias de prueba
memorias_prueba = [
    "La capital de Francia es Paris.",
    "Trabajo en IA y me interesa NLP y sistemas RAG.",
    "Me gusta el café por la mañana y suelo tomar un espresso doble."
]

print("=== PRUEBAS SOBRE AGENTE Memory_Agent (con deduplicado en add_memory) ===\n")

# 1) Guardar memorias a través del agente
print("1) Guardando memorias mediante el agente:")
for texto in memorias_prueba:
    try:
        prompt = f"Guarda: {texto}"
        out = call_agent(memory_agent, prompt)
        print(f" - Prompt: {prompt}")
        print("   Agent ->", out)
        time.sleep(0.2)
    except Exception:
        print("   Error invocando agente para guardar (ver traceback):")
        traceback.print_exc()

# 2) Intentar guardar duplicados (misma consulta exacta)
print("\n2) Intentando guardar duplicados (debe informar 'Memory already exists'):")
for texto in memorias_prueba:
    try:
        prompt = f"Guarda: {texto}"  # misma consulta
        out = call_agent(memory_agent, prompt)
        print(f" - Prompt: {prompt}")
        print("   Agent ->", out)
        time.sleep(0.1)
    except Exception:
        print("   Error invocando agente para duplicado (ver traceback):")
        traceback.print_exc()

# 3) Recuperar con prompts naturales (el agente debe usar search_memory)
print("\n3) Recuperación usando el agente (prompts naturales):")
queries = [
    "¿Cuál es la capital de Francia?",
    "¿Qué dije sobre mi trabajo en IA?",
    "¿Qué dije sobre el café?"
]
for q in queries:
    try:
        print(f"\n - Prompt agente: {q}")
        agent_resp = call_agent(memory_agent, q)
        print("   Agent ->", agent_resp)
        # Mostrar low-level matches para verificación
        try:
            low = search_and_fetch(q, top_k=5)
            print("   Low-level matches:")
            if not low:
                print("    (sin resultados)")
            else:
                for i, r in enumerate(low, 1):
                    meta = r.get("metadata") or {}
                    text = meta.get("text") or meta.get("snippet") or "<sin text>"
                    print(f"    {i}) score={r.get('score')} | id={r.get('id')} | text: {text}")
        except Exception:
            print("   Error en search_and_fetch (ver traceback):")
            traceback.print_exc()
    except Exception:
        print("   Error invocando agente (ver traceback):")
        traceback.print_exc()

print("\n=== FIN PRUEBAS AGENTE Memory_Agent ===")

## Parte 5: Creación de un agente de investigación con la API de Wikipedia y el patrón ReAct

Implementar un agente de búsqueda que utilice la API de Wikipedia para responder preguntas o investigar temas de interés.
1. Crea una instancia del conector de Wikipedia y define la herramienta de búsqueda.
2. Crea un mensaje de sistema (prompt) que describa el rol del agente.
3. Agrupa las herramientas en una lista.
4. Crea el agente de investigación (Research Agent) utilizando el patrón ReAct, mediante la función create_react_agent().

In [ ]:
from langchain_core.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langgraph.prebuilt.chat_agent_executor import create_react_agent
from langgraph.prebuilt import ToolNode

# Aseguramos que exista el modelo de lenguaje
llm_model = globals().get("llm_model") or globals().get("chat")
if llm_model is None:
    raise RuntimeError("No se encontró 'llm_model' ni 'chat'. Inicialízalo antes de crear el Research_Agent.")

# Conector de Wikipedia (en español, puedes cambiar lang="es" si lo prefieres en otro idioma)
api_wrapper = WikipediaAPIWrapper(lang="es", doc_content_chars_max=2000)

# Herramienta de búsqueda en Wikipedia
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Prompt del sistema para el agente de investigación
search_agent_prompt = (
    "Eres Research_Agent. Tu objetivo es investigar temas utilizando la API de Wikipedia.\n"
    "Dispones de una herramienta llamada 'wikipedia' que consulta Wikipedia a partir de una consulta de texto.\n\n"
    "Instrucciones:\n"
    " - Cuando el usuario haga una pregunta sobre un concepto, evento, persona o tema de interés, "
    "   usa la herramienta de Wikipedia para obtener información.\n"
    " - Integra y resume la información encontrada y responde de forma clara, concisa y en español.\n"
    " - Cita brevemente las secciones relevantes cuando sea útil para el usuario.\n"
)

# Lista de herramientas del agente de investigación
tools_list = [wikipedia_tool]
tools_node = ToolNode(tools_list)

# Creación del agente de investigación usando el patrón ReAct
research_agent = create_react_agent(
    llm_model,
    tools_node,
    prompt=search_agent_prompt,
    name="Research_Agent"
)

# Ensure the name attribute exists and is correct for the supervisor
if not hasattr(research_agent, 'name') or research_agent.name is None or research_agent.name == "LangGraph":
    print(f"Warning: Setting 'name' attribute on research_agent to 'Research_Agent'.")
    research_agent.name = "Research_Agent"


print("Agente 'Research_Agent' creado correctamente.")

Ejemplo de uso (ejecutalo tras definir la función):

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
import time, traceback

print("=== PRUEBAS SOBRE Research_Agent (Wikipedia) ===\n")

queries = [
    "¿Quién fue Alan Turing?",
    "Explica brevemente qué es el aprendizaje profundo.",
    "¿Qué es la teoría de la relatividad?",
]

for q in queries:
    print(f"Pregunta: {q}")
    try:
        state_in = {"messages": [HumanMessage(content=q)]}
        result_state = research_agent.invoke(state_in)

        msgs = result_state["messages"]
        ai_msgs = [m for m in msgs if isinstance(m, AIMessage)]
        answer = ai_msgs[-1].content if ai_msgs else msgs[-1].content

        print(f"Respuesta del agente:\n\n{answer}")
    except Exception:
        print("  Error invocando Research_Agent (ver traceback):")
        traceback.print_exc()

    print("\n" + "-" * 60 + "\n")
    time.sleep(0.2)

print("=== FIN PRUEBAS Research_Agent ===")

## Parte 6: Creación de un agente supervisor para coordinar los agentes de memoria e investigación

Implementar un agente supervisor que dirija dinámicamente las solicitudes del usuario hacia el agente más adecuado (ya sea el de memoria o el de investigación).
1. Define el prompt del supervisor, que describa su rol de coordinación
2. Crea el supervisor utilizando la función create_supervisor(), especificando:
    - La lista de agentes que supervisará: [research_agent, memory_agent].
    - El modelo de lenguaje (llm_model).
    - El prompt del sistema (supervisor_agent_prompt).
    - El modo de salida (output_mode="full_history") para conservar el registro completo de la interacción.
3. Compila el agente supervisor.


In [ ]:
from langgraph_supervisor import create_supervisor

# Prompt del supervisor: coordina Memory_Agent y Research_Agent
supervisor_agent_prompt = (
    "Eres un agente supervisor que coordina dos agentes especializados:\n"
    "- Memory_Agent: gestiona memorias personales del usuario (guardar y recuperar información propia).\n"
    "- Research_Agent: investiga temas generales utilizando Wikipedia.\n\n"
    "Instrucciones:\n"
    "- Si la consulta del usuario se refiere a información personal, recuerdos, notas o algo que debería guardarse "
    "  o recuperarse más tarde, delega en Memory_Agent.\n"
    "- Si la consulta requiere conocimiento enciclopédico, hechos históricos, definiciones o investigación general, "
    "  delega en Research_Agent.\n"
    "- Asigna trabajo a un solo agente por turno y no hagas tú mismo el trabajo; tu rol es sólo decidir y orquestar.\n"
    "- Devuelve siempre al usuario una respuesta clara y concisa en español, basada en el resultado del agente elegido.\n\n"
    # ---- Instrucción añadida para testing automático ----
    "IMPORTANTE PARA TESTS AUTOMATIZADOS: además de la respuesta natural al usuario, "
    "al final añade en una línea separada un JSON válido con los campos "
    "`delegated_agent` y `tool_used`. Ejemplo:\n"
    '{"delegated_agent": "Memory_Agent", "tool_used": "memory.save"}\n'
    "Este JSON debe ser en una sola línea y válido para parsear por el test.\n"
    # ----------------------------------------------------
)


# Crear el supervisor (workflow) que coordina research_agent y memory_agent
supervisor = create_supervisor(
    [research_agent, memory_agent],
    model=llm_model,
    prompt=supervisor_agent_prompt,
    output_mode="full_history",  # conservar todo el historial de la interacción
)

# Compilar el agente supervisor listo para invocarse
supervisor_agent = supervisor.compile(name="Supervisor_Agent")

Ejemplo de uso (ejecutalo tras definir la función):

In [ ]:
import time
import traceback
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

print("=== PRUEBAS SOBRE Supervisor_Agent ===\n")

# Casos de prueba: algunos deberían ir a Memory_Agent y otros a Research_Agent
pruebas = [
    ("Memoria - guardar",   "Guarda: Mi película favorita es Matrix."),
    ("Memoria - recuperar", "¿Qué dije sobre mi película favorita?"),
    ("Investigación",       "¿Quién fue Alan Turing?"),
    ("Investigación",       "Explica brevemente qué es el aprendizaje profundo."),
]

for etiqueta, prompt in pruebas:
    print(f"[Caso: {etiqueta}]")
    print(f"Prompt del usuario: {prompt}")
    try:
        # Estado inicial para el supervisor
        state_in = {
            "messages": [HumanMessage(content=prompt)]
        }

        # Invocar al agente supervisor
        result_state = supervisor_agent.invoke(state_in)

        # El supervisor fue creado con output_mode="full_history",
        # así que result_state["messages"] incluye toda la conversación
        msgs = result_state["messages"]

        # 1) detectar qué agente se invocó (último AIMessage)
        ai_msgs = [m for m in msgs if isinstance(m, AIMessage)]
        if ai_msgs:
            last_ai = ai_msgs[-1]
        else:
            last_ai = msgs[-1]

        # Nombre del agente que generó la respuesta (si está disponible)
        agent_name = getattr(last_ai, "name", "desconocido")

        # 2) detectar qué herramienta se ejecutó (último ToolMessage)
        tool_msgs = [m for m in msgs if isinstance(m, ToolMessage)]
        last_tool = tool_msgs[-1].name if tool_msgs else "ninguna"

        print(f"Agente delegado: {agent_name}")
        print(f"Herramienta usada: {last_tool}")
        print("Respuesta del agente:\n" + last_ai.content)

    except Exception:
        print("  Error invocando Supervisor_Agent (ver traceback):")
        traceback.print_exc()

    print("\n" + "-" * 60 + "\n")
    time.sleep(0.2)

print("=== FIN PRUEBAS Supervisor_Agent ===")

## Parte 7: Construcción del grafo de orquestación con LangGraph

Diseñar y compilar un grafo funcional que orqueste la interacción entre el supervisor y los agentes subordinados (investigador y memoria), utilizando LangGraph.
1. Crea una instancia del grafo
2. Agrega los nodos y aristas correspondientes
3. Compila el grafo final

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import re # Import the re module for regular expressions

# 1. Crea una instancia del grafo
# Define el estado del grafo. En este caso, solo necesitamos el historial de mensajes.
# LangGraph ya tiene un tipo de estado predefinido para esto: MessagesState
workflow = StateGraph(MessagesState)

# 2. Agrega los nodos y aristas correspondientes

# Nodos para los agentes y el supervisor
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("research_agent", research_agent)
workflow.add_node("memory_agent", memory_agent)

# Define el punto de entrada
workflow.add_edge(START, "supervisor")

# Define la función de enrutamiento basada en la decisión del supervisor
# El supervisor debe devolver un nombre de agente válido ("research_agent" o "memory_agent")
def route_agents(state):
    # El último mensaje debe ser del supervisor y contener la decisión.
    decision_raw = state["messages"][-1].content.strip()
    print(f"Supervisor decidió (raw): {decision_raw}")

    # Try to extract the agent name by looking for specific patterns in the output
    decision = None
    match = re.search(r"Successfully transferred to (Memory_Agent|Research_Agent)", decision_raw)
    if match:
        decision = match.group(1).lower() # Extract the agent name and convert to lowercase

    if decision is None:
        # Fallback or error handling if decision is unclear
        print(f"Advertencia: Decisión del supervisor no clara o formato inesperado: '{decision_raw}'. Volviendo al supervisor.")
        decision = "supervisor" # Or handle as an error/default

    print(f"Supervisor decidió (enrutado): {decision}")
    return decision

# Agrega la arista condicional desde el supervisor
# La arista va del supervisor a la función de enrutamiento.
# La función de enrutamiento decide a dónde ir a continuación.
workflow.add_conditional_edges(
    "supervisor", # Desde el nodo supervisor
    route_agents, # Función que decide a dónde ir
    {
        "research_agent": "research_agent", # Si route_agents devuelve "research_agent", ir al nodo research_agent
        "memory_agent": "memory_agent",   # Si route_agents devuelve "memory_agent", ir al nodo memory_agent
        "supervisor": "supervisor" # Added fallback to supervisor
    }
)

# Los agentes (memory_agent y research_agent) siempre terminan su ejecución
# y devuelven el control al supervisor (o al final del grafo, dependiendo del diseño)
# Para este ejemplo simple, asumimos que después de que un agente responde, el proceso termina.
workflow.add_edge("research_agent", END)
workflow.add_edge("memory_agent", END)


# 3. Compila el grafo final
graph = workflow.compile()

print("Grafo funcional compilado correctamente.")

Pruebas para validar grafo

- Modo conciso: imprime solo la etiqueta del caso, el agente delegado (si se detecta) y la respuesta final:



In [ ]:
import time, json, re, traceback
from langchain_core.messages import HumanMessage, AIMessage

pruebas = [
    ("Memoria - guardar",   "Guarda: Mi película favorita es Matrix."),
    ("Memoria - recuperar", "¿Qué dije sobre mi película favorita?"),
    ("Investigación",       "¿Quién fue Alan Turing?"),
    ("Investigación",       "Explica brevemente qué es el aprendizaje profundo."),
]

# patrón para detectar mensajes de hand-off cortos (evitarlos)
handoff_re = re.compile(r'transfer(back|ring)?|transferri?ng back|transferido al supervisor', re.IGNORECASE)

def find_meta(msgs):
    for m in reversed(msgs):
        if isinstance(m, AIMessage):
            j = re.search(r'(\{(?:.|\s)*\})\s*$', (m.content or "").strip(), re.DOTALL)
            if j:
                try:
                    meta = json.loads(j.group(1))
                    if isinstance(meta, dict) and "delegated_agent" in meta:
                        return meta
                except Exception:
                    pass
    return None

def select_response(msgs):
    # 1) si hay metadata con delegated_agent -> elegir AIMessage de ese agente más representativo
    meta = find_meta(msgs)
    if meta and meta.get("delegated_agent"):
        delegated = meta["delegated_agent"].lower()
        candidates = [(i, m) for i, m in enumerate(msgs)
                      if isinstance(m, AIMessage) and (getattr(m, "name", "") or "").lower() == delegated]
        if candidates:
            filtered = [(i,m) for i,m in candidates if not (len((m.content or "").strip())<80 and handoff_re.search(m.content or ""))]
            pool = filtered or candidates
            idx, chosen = max(pool, key=lambda im: len((im[1].content or "")))
            return chosen, meta.get("delegated_agent")
    # 2) si no hay metadata, escoger AIMessage con nombre de agente conocido (si existe) o el AIMessage más largo
    known_agent_names_lower = {"memory_agent", "memory_agent".lower(), "research_agent", "research_agent".lower(),
                               "memory_agent", "research_agent", "supervisor", "supervisor_agent"}
    ai_msgs = [(i, m) for i, m in enumerate(msgs) if isinstance(m, AIMessage)]
    by_name = [(i,m) for i,m in ai_msgs if (getattr(m, "name", "") or "").lower() in known_agent_names_lower]
    if by_name:
        idx, chosen = max(by_name, key=lambda im: len((im[1].content or "")))
        return chosen, getattr(chosen, "name", None)
    if ai_msgs:
        idx, chosen = max(ai_msgs, key=lambda im: len((im[1].content or "")))
        return chosen, getattr(chosen, "name", None)
    return None, None

for etiqueta, prompt in pruebas:
    try:
        state_in = {"messages": [HumanMessage(content=prompt)]}
        invoker = supervisor_agent if 'supervisor_agent' in globals() else (graph if 'graph' in globals() else None)
        if invoker is None:
            raise RuntimeError("No se encontró 'supervisor_agent' ni 'graph' en el entorno.")
        result = invoker.invoke(state_in)
        msgs = result.get("messages", [])
        chosen, agent = select_response(msgs)
        respuesta = (chosen.content or "").strip() if chosen else "(sin respuesta identificada)"
        agente_label = agent or "desconocido"
        print(f"[{etiqueta}]")
        print(f"Agente delegado: {agente_label}")
        print("Respuesta:\n" + respuesta + "\n")
    except Exception:
        print(f"[{etiqueta}] Error ejecutando la prueba:")
        traceback.print_exc()
    time.sleep(0.12)


- Modo de debug: imprime historial, metadata, motivo de selección y el mensaje completo del agente delegado:

In [ ]:
import time
import traceback
import json
import re
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

print("=== PRUEBAS SOBRE Supervisor_Agent (mejoradas, corrección selección Research_Agent) ===\n")

# Casos de prueba
pruebas = [
    ("Memoria - guardar",   "Guarda: Mi película favorita es Matrix."),
    ("Memoria - recuperar", "¿Qué dije sobre mi película favorita?"),
    ("Investigación",       "¿Quién fue Alan Turing?"),
    ("Investigación",       "Explica brevemente qué es el aprendizaje profundo."),
]

# Detectar nombres conocidos (intenta leer .name si existe)
known_agent_names = set()
try:
    if 'memory_agent' in globals() and hasattr(memory_agent, 'name'):
        known_agent_names.add(getattr(memory_agent, 'name'))
    if 'research_agent' in globals() and hasattr(research_agent, 'name'):
        known_agent_names.add(getattr(research_agent, 'name'))
except Exception:
    pass

known_agent_names.update({
    "Memory_Agent", "memory_agent", "MemoryAgent", "memoryagent",
    "Research_Agent", "research_agent", "ResearchAgent", "researchagent",
    "Supervisor_Agent", "supervisor", "Supervisor"
})
known_agent_names_lower = {n.lower() for n in known_agent_names}

def dump_history(msgs):
    print(">> Historial completo (índice, tipo, name/role, len):")
    for i, m in enumerate(msgs):
        typ = type(m).__name__
        name = getattr(m, "name", getattr(m, "role", "<sin nombre>"))
        content = getattr(m, "content", "")
        snippet = content if len(content) <= 220 else content[:220] + "...(truncado)"
        print(f"  [{i:02d}] {typ:12} | name/role='{name}' | len={len(content):4} | snippet: {snippet!r}")
    print("")

def find_json_metadata_in_messages(msgs):
    for i, m in reversed(list(enumerate(msgs))):
        if isinstance(m, AIMessage):
            text = m.content.strip()
            jmatch = re.search(r'(\{(?:.|\s)*\})\s*$', text)
            if jmatch:
                try:
                    meta = json.loads(jmatch.group(1))
                    if isinstance(meta, dict) and ("delegated_agent" in meta or "tool_used" in meta):
                        return meta, i, m
                except Exception:
                    continue
    return None, None, None

# patrón para detectar mensajes de hand-off cortos (en varios idiomas/frases comunes)
handoff_pattern = re.compile(r'(transfer(back|ring)?\s*(to|back)?\s*supervisor|transferri?ng back to supervisor|successfully transferred back|transferido al supervisor)', re.IGNORECASE)

def pick_agent_response(msgs):
    """
    Mejor lógica:
    - Si hay JSON metadata -> buscar TODOS los AIMessage con name==delegated_agent y elegir el más largo
      (ignorar mensajes cortos / de handoff mediante patrón)
    - Si no hay metadata, aplicar heurísticas previas (por nombre conocido, tool messages, o AI más largo).
    Devuelve: (chosen_message, chosen_index, reason, meta)
    """
    meta, meta_idx, meta_msg = find_json_metadata_in_messages(msgs)
    if meta:
        delegated = meta.get("delegated_agent")
        tool_used = meta.get("tool_used")
        reason = f"Encontrado JSON metadata en AIMessage índice {meta_idx} (delegated_agent={delegated}, tool_used={tool_used})"

        # Buscar todas las AIMessage cuyo .name coincide con delegated (case-insensitive)
        candidates = []
        if delegated:
            for i, m in enumerate(msgs):
                if isinstance(m, AIMessage) and getattr(m, "name", "").lower() == delegated.lower():
                    candidates.append((i, m))

            if candidates:
                # Filtrar mensajes claramente de handoff o extremadamente cortos
                filtered = []
                for i, m in candidates:
                    text = (m.content or "").strip()
                    if len(text) < 50 and handoff_pattern.search(text):
                        # descartamos evidentes handoffs cortos
                        continue
                    filtered.append((i, m))

                # Si el filtrado deja vacíos, usamos los candidatos originales (para no perder todo)
                pool = filtered if filtered else candidates

                # Elegir el de mayor longitud de contenido
                chosen_idx, chosen_msg = max(pool, key=lambda im: len((im[1].content or "")))
                reason += f" -> {len(pool)} candidato(s) encontrados, seleccionado índice {chosen_idx} por mayor longitud"
                return chosen_msg, chosen_idx, reason, meta

            else:
                reason += " -> NO se encontraron AIMessage con el nombre delegado en el historial."
                # continuamos con heurísticas fallback

    # ---------------- Fallbacks (sin metadata o metadata inconclusa) ----------------
    # 1) AIMessages con nombre de agente conocido (preferir más largos)
    ai_msgs = [(i, m) for i, m in enumerate(msgs) if isinstance(m, AIMessage)]
    candidates_by_name = [(i, m) for i, m in ai_msgs if getattr(m, "name", "").lower() in known_agent_names_lower]
    if candidates_by_name:
        chosen_idx, chosen_msg = max(candidates_by_name, key=lambda im: len((im[1].content or "")))
        reason = "Cruce por nombre: elegido AIMessage (agente conocido) con mayor longitud"
        return chosen_msg, chosen_idx, reason, meta

    # 2) ToolMessage significativa -> buscar AIMessage cerca
    tool_msgs = [(i, m) for i, m in enumerate(msgs) if isinstance(m, ToolMessage)]
    meaningful_tools = [(i, m) for i, m in tool_msgs if m.name and "transfer_back_to_supervisor" not in m.name.lower()]
    if meaningful_tools:
        idx, tool_m = meaningful_tools[-1]
        after_msgs = [(i, m) for i, m in enumerate(msgs[idx+1:], start=idx+1) if isinstance(m, AIMessage)]
        if after_msgs:
            chosen_idx, chosen_msg = after_msgs[0]
            reason = f"Encontrada ToolMessage significativa '{tool_m.name}' (índice {idx}) -> tomando AIMessage siguiente (índice {chosen_idx})"
            return chosen_msg, chosen_idx, reason, meta
        before_msgs = [(i, m) for i, m in enumerate(msgs[:idx]) if isinstance(m, AIMessage)]
        if before_msgs:
            chosen_idx, chosen_msg = before_msgs[-1]
            reason = f"Encontrada ToolMessage significativa '{tool_m.name}' (índice {idx}) -> tomando AIMessage anterior (índice {chosen_idx})"
            return chosen_msg, chosen_idx, reason, meta

    # 3) Si hay AIMessages múltiples, preferir el más largo
    if ai_msgs:
        chosen_idx, chosen_msg = max(ai_msgs, key=lambda im: len((im[1].content or "")))
        reason = "Fallback: elegido AIMessage más largo del historial"
        return chosen_msg, chosen_idx, reason, meta

    return None, None, "No se halló AIMessage en el historial", meta

# Ejecutar pruebas
for etiqueta, prompt in pruebas:
    print(f"[Caso: {etiqueta}]")
    print(f"Prompt del usuario: {prompt}")
    try:
        state_in = {"messages": [HumanMessage(content=prompt)]}

        # elegir invocador (supervisor_agent o graph)
        if 'supervisor_agent' in globals():
            invoker = supervisor_agent
            invoker_name = "supervisor_agent"
        elif 'graph' in globals():
            invoker = graph
            invoker_name = "graph"
        else:
            raise RuntimeError("No se encontró 'supervisor_agent' ni 'graph' en el entorno.")

        print(f"  Paso 1: invocando {invoker_name}.invoke(...)")
        result_state = invoker.invoke(state_in)
        print("  Paso 2: invocación completada, extrayendo full_history\n")

        msgs = result_state.get("messages", [])
        dump_history(msgs)

        chosen_msg, chosen_idx, reason, meta = pick_agent_response(msgs)

        # Mostrar info adicional sobre la tool wikipedia si existe
        tool_msgs_all = [(i, m) for i, m in enumerate(msgs) if isinstance(m, ToolMessage)]
        wiki_tool = next(((i, m) for i, m in tool_msgs_all if getattr(m, "name", "").lower() in {"wikipedia", "wiki"}), None)
        if wiki_tool:
            i_w, m_w = wiki_tool
            print(f"Se detectó ToolMessage de Wikipedia en índice {i_w}, longitud {len(m_w.content or '')} caracteres.")

        if chosen_msg:
            agent_name = getattr(chosen_msg, "name", "desconocido")
            content_full = getattr(chosen_msg, "content", "")
            print("======== Resultado detectado ========")
            print(f"Motivo de selección: {reason}")
            if meta:
                print(f"Metadata JSON detectada: {meta}")
            print(f"Índice del AIMessage seleccionado: {chosen_idx}")
            print(f"Agente declarado en el AIMessage seleccionado: '{agent_name}'")
            print(f"Longitud de contenido seleccionada: {len(content_full)} caracteres")
            print("\n--- Contenido completo del mensaje seleccionado ---\n")
            print(content_full)
            print("\n--- fin del contenido ---\n")
        else:
            print("NO se pudo identificar un AIMessage representativo del agente delegado.")
            if tool_msgs_all:
                last_tool = tool_msgs_all[-1][1]
                print(f"Última ToolMessage encontrada: name='{last_tool.name}', content_len={len(getattr(last_tool, 'content', ''))}")
            print("Historial completo arriba para debug.")

    except Exception:
        print("  Error invocando Supervisor/Graph (ver traceback):")
        traceback.print_exc()

    print("\n" + "-" * 80 + "\n")
    time.sleep(0.2)

print("=== FIN PRUEBAS Supervisor_Agent ===")
